In [1]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median
from collections import Counter

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
lr = 1e-3
env = gym.make('CartPole-v0')
env.reset()
goal_steps = 500
score_requirement = 50
initial_games = 10000

[2017-09-02 16:10:56,015] Making new env: CartPole-v0


In [3]:
def some_random_games_first():
    for episode in range(5):
        env.reset()
        for t in range(goal_steps):
            env.render()
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if done:
                break
                
# some_random_games_first()

In [4]:
# To get training data
def initial_population():
    training_data = []
    scores = []
    accepted_scores = []
    for _ in range(initial_games):
        score = 0
        game_memory = []
        prev_observation = []
        for _ in range(goal_steps):
            action = random.randrange(0, 2)
            observation, reward, done, info = env.step(action)
            
            if len(prev_observation) > 0:
                game_memory.append([prev_observation, action])
                
            prev_observation = observation
            score += reward
            if done:
                break
        
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1]
                    
                elif data[1] == 0:
                    output = [1, 0]
                    
                training_data.append([data[0], output])
                
        env.reset()
        scores.append(score)
    training_data_save = np.array(training_data)
    np.save('saved.npy', training_data_save)
    print('Average accepted score: ', mean(accepted_scores))
    print('Median accepted score: ', median(accepted_scores))
    print(Counter(accepted_scores))
    return training_data

In [5]:
initial_population()

Average accepted score:  61.15064935064935
Median accepted score:  58.0
Counter({50.0: 35, 51.0: 35, 54.0: 26, 52.0: 22, 53.0: 21, 63.0: 19, 56.0: 18, 61.0: 16, 55.0: 16, 59.0: 14, 57.0: 13, 60.0: 13, 58.0: 12, 65.0: 11, 70.0: 10, 62.0: 9, 64.0: 8, 68.0: 8, 71.0: 7, 66.0: 7, 69.0: 6, 67.0: 6, 72.0: 6, 74.0: 5, 77.0: 5, 75.0: 4, 76.0: 3, 78.0: 3, 82.0: 2, 80.0: 2, 84.0: 2, 92.0: 2, 103.0: 2, 95.0: 2, 79.0: 2, 96.0: 1, 108.0: 1, 127.0: 1, 100.0: 1, 98.0: 1, 107.0: 1, 97.0: 1, 89.0: 1, 83.0: 1, 73.0: 1, 106.0: 1, 86.0: 1, 90.0: 1})


[[array([-0.02417875,  0.21714328,  0.03949008, -0.24827456]), [0, 1]],
 [array([-0.01983589,  0.41167966,  0.03452459, -0.52824457]), [1, 0]],
 [array([-0.01160229,  0.21608942,  0.02395969, -0.22488583]), [1, 0]],
 [array([-0.00728051,  0.02063338,  0.01946198,  0.07525757]), [0, 1]],
 [array([-0.00686784,  0.21547099,  0.02096713, -0.21122205]), [1, 0]],
 [array([-0.00255842,  0.02005563,  0.01674269,  0.08800046]), [0, 1]],
 [array([-0.00215731,  0.21493364,  0.0185027 , -0.19935347]), [0, 1]],
 [array([ 0.00214137,  0.40978613,  0.01451563, -0.4861427 ]), [1, 0]],
 [array([ 0.01033709,  0.21446239,  0.00479277, -0.18892051]), [1, 0]],
 [array([ 0.01462634,  0.0192722 ,  0.00101436,  0.10527048]), [0, 1]],
 [array([ 0.01501178,  0.2143796 ,  0.00311977, -0.18709224]), [1, 0]],
 [array([ 0.01929937,  0.01921315, -0.00062207,  0.10657322]), [0, 1]],
 [array([ 0.01968364,  0.21434401,  0.00150939, -0.18630591]), [1, 0]],
 [array([ 0.02397052,  0.0192005 , -0.00221672,  0.10685279]), [

In [6]:
def neural_network_model(input_size):
    network = input_data(shape=[None, input_size, 1], name='input')
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=lr, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(network)
    return model

In [7]:
def train_model(training_data, model=False):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]), 1)
    y = [i[1] for i in training_data]
    
    if not model:
        model = neural_network_model(input_size=len(X[0]))
        
    model.fit({'input': X}, {'targets': y}, n_epoch=5, snapshot_step=500, show_metric=True, run_id='openai_cart_pole_v0')
    return model

In [8]:
training_data = initial_population()
model = train_model(training_data)

Training Step: 1624  | total loss: 0.66930 | time: 6.678s
| Adam | epoch: 005 | loss: 0.66930 - acc: 0.5922 -- iter: 20736/20784
Training Step: 1625  | total loss: 0.67003 | time: 6.694s
| Adam | epoch: 005 | loss: 0.67003 - acc: 0.5986 -- iter: 20784/20784
--


In [9]:
scores = []
choices = []

for each_game in range(10):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    for _ in range(goal_steps):
        env.render()
        if len(prev_obs) == 0:
            action = random.randrange(0, 2)
        else:
            action = np.argmax(model.predict(prev_obs.reshape(-1, len(prev_obs), 1))[0])
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        game_memory.append([new_observation, action])
        score += reward
        if done:
            break
        
    scores.append(score)

print('Average Score', sum(scores)/len(scores))
print('Choice 1: {}, Choice 0: {}'.format(choices.count(1)/len(choices), choices.count(0)/len(choices)))

NameError: name 'rangom' is not defined